##Activación

Activar sincronizacion con Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Nos movemos al directorio que contiene el proyecto

In [2]:
%cd /content/drive/My Drive/Proyecto-Inteligencia/03-Prototipo
!pwd

/content/drive/My Drive/Proyecto-Inteligencia/03-Prototipo
/content/drive/My Drive/Proyecto-Inteligencia/03-Prototipo


##Importacion

Importamos librerias necesarias para la ejecución del proyecto 

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.decomposition import PCA

Mostramos la información del dataset que vamos a tratar:

In [0]:
inf = pd.read_csv('../data/csv/Descripcion_campos.csv')
print(inf)

Importamos dataset

In [0]:
df = pd.read_csv('../data/csv/rain2019.csv')
df.info() #Despliega la informacion del dataset
df.head() #Despliega la cabecera del dataset

In [0]:
df.hist(bins=15, figsize=(10,10))

##Preprocesamiento de datos

Eliminamos las columnas que no nos son de utilidad para limpiar el dataset y reindexamos por fecha

In [0]:
print(type(df))
del df['cod_est']
del df['V_Min']
del df['SumaDeLluvia']
del df['SumaDeCV1']
del df['SumaDeCV2']
del df['SumaDeCV3']
del df['SumaDeCV4']
df['FECHA'] = pd.to_datetime(df['FECHA']) #Se convierte la fecha a formato de tiempo
df.set_index('FECHA', inplace=True) #Utilizamos la fecha como indice
df.dropna(inplace = True) #Elimina campos vacios
df.head()

##Despliegue grafico

Hacemos el histograma de cada columna

In [0]:
df.hist(bins=15, figsize=(10,10))

Miramos la correlación que tienen las columnas del dataset

In [0]:
pd.plotting.scatter_matrix(df.sample(1500), figsize=(10,10));

#Procesamiento de datos

Pasamos el dataframe a un arreglo Numpy, lo permutamos para asegurarnos de tomar muestras aleatorias y definimos nuestras características (X) y etiquetas (y).

In [0]:
val = np.random.permutation(df.values)
X = val[:,[0,1,2,3,4,5,6,8,9]]
y = val[:,6]
print(X.shape)
print(type(X))
print(y.shape)
print(type(y))


Miramos cuales son los valores maximos y minimos de cada columna.

In [0]:
maximo = df.max(axis = 0 )
minimo = df.min(axis = 0)
print('Maximos de cada columna:\n',maximo)
print('\nMinimos de cada columna:\n',minimo)

**PCA**

In [0]:
#X = PCA(n_components=4).fit_transform(X)

Definimos nuestros valores para entrenamiento y prueba, y el tamaño de cada uno

In [0]:
X = X.astype('float32')#Lo pasamos a flotantes
y = y.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

#Machine Learning

## 1. Random Forest:


Vamos a medir cual sería la mejor configuracion de profundidad para éste estimador

In [0]:
def show_curve_RFR(nfold, nTree_Range):
    means, stds = [], []
    si = 1
    for nTree in nTree_Range:
        print (nTree),
        s = cross_val_score(RandomForestRegressor(max_depth=nTree), X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
        means.append(np.mean(s))
        stds.append(np.std(s))
        if np.mean(s)<si:
          BestTree = nTree
          si = np.mean(s)   

    means = np.r_[means]
    stds  = np.r_[stds]

    plt.plot(nTree_Range, means, label="mean", color="black")
    plt.fill_between(nTree_Range, means-stds, means+stds, color="blue", alpha=.5, label="std")
    plt.xlabel("n Tree")
    plt.ylabel("performance")
    plt.legend()

    return BestTree

In [0]:
nfold = 10
nTree_Range = range(1,10)
BestTree = show_curve_RFR(nfold, nTree_Range)
print ("La mejor configuración para la profundidad del arbol es: ", BestTree)

Definir un objeto de tipo RandomForestRegressor

In [0]:
est_RF= RandomForestRegressor()
s = cross_val_score(est_RF, X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
print ("MSE %.6f (+/- %.5f)"%(np.mean(s), np.std(s)))

Entrenamos nuestro modelo, y hacemos las predicciones

In [0]:
est_RF.fit(X_train, y_train)
predic_RF = est_RF.predict(X_test)

Graficamos las predicciones con el test, para saber el nivel de precisión de nuestro modelo.

In [0]:
plt.scatter(X_test[:,2], y_test, color = "red")
plt.scatter(X_test[:,2], predic_RF, color = "green", alpha = .3)
plt.grid()
plt.legend(labels=['Test','Predic'])
plt.title("T media vs prom_rad")
plt.xlabel("T  media")
plt.ylabel("prom_rad")
plt.show()

## 2. Decision tree:

Vamos a medir cual sería la mejor configuracion de profundidad para éste estimador

In [0]:
def show_curve_DTR(nfold, nTree_Range):
    means, stds = [], []
    si = 1
    for nTree in nTree_Range:
        s = cross_val_score(DecisionTreeRegressor(max_depth=nTree), X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
        means.append(np.mean(s))
        stds.append(np.std(s))
        if np.mean(s)<si:
          BestTree = nTree
          si = np.mean(s)        
    means = np.r_[means]
    stds  = np.r_[stds]

    plt.plot(nTree_Range, means, label="mean", color="black")
    plt.fill_between(nTree_Range, means-stds, means+stds, color="blue", alpha=.5, label="std")
    plt.xlabel("n Tree")
    plt.ylabel("performance")
    plt.legend()

    return BestTree

In [0]:
nfold = 10
nTree_Range = range(10,50)
BestTree = show_curve_DTR(nfold, nTree_Range)
print ("La mejor configuración para la profundidad del arbol es: ", BestTree)

Definir un objeto de tipo DecisionTreeRegressor

In [0]:
#Se aplica decision tree
est_DT = DecisionTreeRegressor(max_depth=BestTree)
s = cross_val_score(est_DT, X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
print("MSE %.6f (+/- %.5f)"%(np.mean(s), np.std(s)))

Entrenamos nuestro modelo, y hacemos las predicciones

In [0]:
est_DT.fit(X_train, y_train)
predic_DT = est_DT.predict(X_test)

Graficamos las predicciones con el test, para saber el nivel de precisión de nuestro modelo.

In [0]:
plt.scatter(X_test[:,2], y_test, color = "red")
plt.scatter(X_test[:,2], predic_DT, color = "green", alpha = .3)
plt.grid()
plt.legend(labels=['Test','Predic'])
plt.title("T media vs prom_rad")
plt.xlabel("T  media")
plt.ylabel("prom_rad")
plt.show()

## 3. SVM

Vamos a medir cual sería la mejor configuracion de maximo de iteraciones para éste estimador.

In [0]:
def show_curve_SVR(nfold, max_iter_range):
    means, stds = [], []
    si=1
    for iter in max_iter_range:
        s = cross_val_score(SVR(max_iter=iter), X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
        means.append(np.mean(s))
        stds.append(np.std(s))
        if np.mean(s)<si:
          BestIter = iter
          si = np.mean(s) 
    means = np.r_[means]
    stds  = np.r_[stds]

    plt.plot(nTree_Range, means, label="mean", color="black")
    plt.fill_between(nTree_Range, means-stds, means+stds, color="blue", alpha=.5, label="std")
    plt.xlabel("n Tree")
    plt.ylabel("performance")
    plt.legend()

    return BestIter

In [0]:
nfold = 10
max_iter_range = range(40,100)
BestIter=show_curve_DTR(nfold, max_iter_range)
print ("La mejor configuracion para el maximo de iteraciones es: ", BestIter)

Definir un objeto de tipo SVM

In [0]:
#Se aplica SVR
est_SV = SVR()
s = cross_val_score(est_SV, X, y, cv=KFold(nfold, shuffle=True), scoring=make_scorer(mean_squared_error))
print("MSE %.6f (+/- %.5f)"%(np.mean(s), np.std(s)))

Entrenamos nuestro modelo, y hacemos las predicciones

In [0]:
est_SV.fit(X_train, y_train)
predic_SV = est_SV.predict(X_test)

Graficamos las predicciones con el test, para saber el nivel de precisión de nuestro modelo.

In [0]:
plt.scatter(X_test[:,2], y_test, color = "red")
plt.scatter(X_test[:,2], predic_SV, color = "green", alpha = .3)
plt.title("T media vs prom_rad")
plt.grid()
plt.legend(labels=['Test','Predic'])
plt.xlabel("T  media")
plt.ylabel("prom_rad")
plt.show()

# 4. Deep learning:

In [0]:
model = keras.models.Sequential([ #keras de modelo secuencial, solo se definen las capas
    keras.layers.InputLayer( input_shape=[9]),#
    keras.layers.Dense(512, activation=tf.nn.relu),#capa escondida
    keras.layers.Dense(256, activation=tf.nn.relu),#capa escondida
    keras.layers.Dense(128, activation=tf.nn.relu),#capa escondida
    keras.layers.Dense(1, activation='linear')#Si se desea que sea regresion la ultima capa es una sola neurona y con activacion linear
])

In [0]:
model.summary()

In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(), #Gradiente estocastico  
              loss='mse',#Regla de minimizacion
              metrics=['mse'])#Mide el mean_squared_error

In [0]:
history = model.fit(X_train, y_train, epochs=15,validation_data=(X_test, y_test) ,verbose = 1)

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)